#1. Load JSON files and normalize into a single DataFrame

In [1]:
files_to_process = ["/content/compoundV2_transactions_ethereum_chunk_0.json"
 ,"/content/compoundV2_transactions_ethereum_chunk_1.json"
 ,"/content/compoundV2_transactions_ethereum_chunk_32.json"]

In [2]:
import json
import pandas as pd
from datetime import datetime
import os
import glob # To find files if needed

# --- Configuration ---
# Assuming files are downloaded to a 'data' subfolder
# DATA_DIR = 'data/'
# Manually identify the 3 largest files or use os.getsize
# Example:
 # Replace with actual paths after downloading

# --- Data Loading Function ---
def load_data(filepath):
    """Loads transaction data from a single JSON file."""
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        print(f"Loaded {filepath}")
        return data
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return None

# --- Data Processing Function ---
def process_raw_data(raw_data_list):
    """Processes raw data from multiple files into a single DataFrame."""
    all_tx = []
    tx_types = ['deposits', 'borrows', 'repayments', 'withdrawals', 'liquidations'] # Adjust based on actual keys

    for data in raw_data_list:
        if data is None:
            continue
        for tx_type, transactions in data.items():
            # Normalize key name if needed (e.g., 'repayments' vs 'repays')
            normalized_tx_type = tx_type.lower().replace('s', '') # Simple normalization
            if 'repayment' in normalized_tx_type: normalized_tx_type = 'repay'
            if 'withdrawal' in normalized_tx_type: normalized_tx_type = 'withdraw'

            if not isinstance(transactions, list):
                print(f"Warning: Expected list for {tx_type}, got {type(transactions)}. Skipping.")
                continue

            for tx in transactions:
                try:
                    record = {
                        'tx_hash': tx.get('hash'),
                        'log_id': tx.get('id'), # Unique identifier for the event log
                        'timestamp': pd.to_datetime(int(tx.get('timestamp')), unit='s'),
                        'tx_type': normalized_tx_type,
                        'wallet_id': tx.get('account', {}).get('id'),
                        'asset_symbol': tx.get('asset', {}).get('symbol'),
                        'asset_id': tx.get('asset', {}).get('id'),
                        'amount': float(tx.get('amount', 0)),
                        'amountUSD': float(tx.get('amountUSD', 0)),
                        'liquidator': None,
                        'liquidated_account': None
                    }

                    if normalized_tx_type == 'liquidation':
                        # Liquidation specific fields
                        record['liquidator'] = tx.get('liquidator', {}).get('id')
                        # The 'account' in a liquidation event is the one being liquidated
                        record['liquidated_account'] = tx.get('account', {}).get('id')
                        # For consistency, let's track activity for both liquidator and liquidated
                        # We'll create two effective records or handle during feature engineering
                        # For now, let's focus on the primary actor involved in the tx log:
                        # If we process by wallet, we need to know WHO initiated what.
                        # Let's keep the 'wallet_id' as the 'account' for now,
                        # and use the 'liquidator' field separately during aggregation.
                        # OR: Create separate entries for liquidator action?
                        # Simpler: Process liquidations separately in feature engineering.
                        # Let's assign the primary 'wallet_id' to the liquidated account here.
                        record['wallet_id'] = record['liquidated_account']


                    # Basic validation
                    if record['wallet_id'] and record['timestamp']:
                        all_tx.append(record)
                    else:
                         print(f"Skipping record due to missing wallet_id or timestamp: {tx.get('id')}")


                except Exception as e:
                    print(f"Error processing record {tx.get('id', 'N/A')} in {tx_type}: {e}")

    df = pd.DataFrame(all_tx)
    df = df.sort_values(by='timestamp').reset_index(drop=True)
    print(f"Processed {len(df)} total transactions.")
    return df

# --- Main Loading Sequence ---
raw_data_loaded = []
for f_path in files_to_process:
    raw_data_loaded.append(load_data(f_path))

# Filter out any None results from failed loads
raw_data_loaded = [d for d in raw_data_loaded if d is not None]

if not raw_data_loaded:
    print("Error: No data loaded. Exiting.")
    exit()

df_tx = process_raw_data(raw_data_loaded)

# --- Initial Data Cleaning ---
df_tx = df_tx.dropna(subset=['wallet_id', 'timestamp', 'amountUSD'])
df_tx = df_tx[df_tx['amountUSD'] >= 0] # Ensure no negative amounts make it through

print("Sample processed data:")
print(df_tx.head())
print("\nTransaction Types Found:")
print(df_tx['tx_type'].value_counts())
print(f"\nNumber of unique wallets: {df_tx['wallet_id'].nunique()}")

Streaming output truncated to the last 5000 lines.
Skipping record due to missing wallet_id or timestamp: 0x0afbada6cb79ac19b7fe74ddb7fa1cc5ea807674740089d289862dc108212282-37
Skipping record due to missing wallet_id or timestamp: 0x0eb51ad904a154523cd942ce1131394f29b7f08afa6237bda4ffb56c3c7fb365-10
Skipping record due to missing wallet_id or timestamp: 0x5f01f0bc33e506b64c8f9ef26ee452740b44845df9141aab24cacf1d67de45db-50
Skipping record due to missing wallet_id or timestamp: 0xcb11522aa279b93e49b0a129133f2ba74b6140c35eafd74a3732e2035aa4e127-452
Skipping record due to missing wallet_id or timestamp: 0x39cbd2f486c8f5effab64004083f74af26ea4097ded45d34f33d7809d9dc56c4-29
Skipping record due to missing wallet_id or timestamp: 0x014907dd3220cb1e3d9987f7987d9751fcf5cdbbc890346c21e9a4a103f1c5c0-10
Skipping record due to missing wallet_id or timestamp: 0x79e2443aba3a3799c0e287c84e4ce41547b878088f7c9fc7f5a37e69dc959d50-53
Skipping record due to missing wallet_id or timestamp: 0xe4c39e5a403fc5bd

In [3]:
import numpy as np

# --- Feature Engineering ---
# Ensure data is sorted by time for sequential calculations if needed
df_tx = df_tx.sort_values(by=['wallet_id', 'timestamp'])

# Group by wallet
wallet_groups = df_tx.groupby('wallet_id')

features = {}

# --- Basic Aggregations ---
features['tx_count'] = wallet_groups.size()
features['unique_assets'] = wallet_groups['asset_symbol'].nunique()
features['first_tx_time'] = wallet_groups['timestamp'].min()
features['last_tx_time'] = wallet_groups['timestamp'].max()
features['account_lifespan_days'] = (features['last_tx_time'] - features['first_tx_time']).dt.total_seconds() / (60 * 60 * 24)


In [4]:
import numpy as np
import pandas as pd


# Calculate basic aggregations
features_agg = wallet_groups.agg(
    tx_count=('tx_hash', 'count'), # Use a non-null column like tx_hash
    unique_assets=('asset_symbol', 'nunique'),
    first_tx_time=('timestamp', 'min'),
    last_tx_time=('timestamp', 'max'),
    mean_tx_usd=('amountUSD', 'mean') # Calculate mean USD across all tx types here
)

# Calculate type-specific counts and sums using pivot_table or groupby().size().unstack()
tx_type_counts = df_tx.groupby(['wallet_id', 'tx_type']).size().unstack(fill_value=0)
tx_type_sums_usd = df_tx.groupby(['wallet_id', 'tx_type'])['amountUSD'].sum().unstack(fill_value=0)

# Rename columns for clarity (e.g., 'deposit' -> 'deposit_count', 'borrow' -> 'borrow_total_usd')
tx_type_counts.columns = [f"{col}_count" for col in tx_type_counts.columns]
tx_type_sums_usd.columns = [f"{col}_total_usd" for col in tx_type_sums_usd.columns]


In [5]:

# Combine aggregated features
df_features = pd.concat([features_agg, tx_type_counts, tx_type_sums_usd], axis=1)

# --- Fill NaNs for wallets that might be missing certain tx types ---
# Important: Ensure all expected columns exist, even if they are all zero
expected_tx_types = ['deposit', 'borrow', 'repay', 'withdraw', 'liquidation'] # Make sure these match your normalized types
for ttype in expected_tx_types:
    count_col = f"{ttype}_count"
    sum_col = f"{ttype}_total_usd"
    if count_col not in df_features.columns:
        df_features[count_col] = 0
    if sum_col not in df_features.columns:
        df_features[sum_col] = 0.0

# Fill any remaining NaNs (e.g., mean_tx_usd for wallets with 0 txns - though should be caught by groupby)
df_features.fillna(0, inplace=True)


In [6]:

# --- Calculate derived features AFTER ensuring columns exist ---

df_features['account_lifespan_days'] = (df_features['last_tx_time'] - df_features['first_tx_time']).dt.total_seconds() / (60 * 60 * 24)
# Ensure lifespan is at least a small number for frequency calculations
df_features['account_lifespan_days'] = df_features['account_lifespan_days'].apply(lambda x: max(x, 1/(24*60))) # Minimum 1 minute lifespan


In [7]:

# --- Liquidation Specific Features ---
# Times wallet WAS liquidated is now correctly calculated as 'liquidation_count' and 'liquidation_total_usd'

# Times wallet ACTED AS liquidator (Needs separate calculation as before)
liquidator_actions = df_tx[df_tx['tx_type'] == 'liquidation'].groupby('liquidator').agg(
    acted_liquidator_count=('tx_hash', 'count'),
    acted_liquidator_total_usd=('amountUSD', 'sum')
)
# Merge liquidator actions
df_features = df_features.merge(liquidator_actions, left_index=True, right_index=True, how='left')
# Fill NaNs for wallets that never acted as liquidators
df_features[['acted_liquidator_count', 'acted_liquidator_total_usd']] = df_features[['acted_liquidator_count', 'acted_liquidator_total_usd']].fillna(0)


In [8]:


# --- Derived Behavioral Features (Now Safe to Calculate Ratios) ---
epsilon = 1e-6 # To avoid division by zero

# Activity / Consistency (Using .apply here is okay as it's complex)
df_features['avg_time_between_tx_hours'] = wallet_groups['timestamp'].apply(
    lambda x: x.diff().mean().total_seconds() / 3600 if len(x) > 1 else 0
).fillna(0)

df_features['std_time_between_tx_hours'] = wallet_groups['timestamp'].apply(
    lambda x: x.diff().std().total_seconds() / 3600 if len(x) > 1 else 0
).fillna(0)

# Repayment Behavior Ratios
df_features['repay_borrow_count_ratio'] = df_features['repay_count'] / (df_features['borrow_count'] + epsilon)
df_features['repay_borrow_usd_ratio'] = df_features['repay_total_usd'] / (df_features['borrow_total_usd'] + epsilon)

# Deposit/Withdrawal Behavior
df_features['deposit_withdraw_usd_ratio'] = df_features['deposit_total_usd'] / (df_features['withdraw_total_usd'] + epsilon)
df_features['net_flow_usd'] = (df_features['deposit_total_usd'] + df_features['repay_total_usd']) - \
                              (df_features['borrow_total_usd'] + df_features['withdraw_total_usd'] + df_features['liquidation_total_usd'])


In [9]:

# Risk Indicators
# !! This is where the original error likely occurred !! Now it should work.
df_features['borrow_deposit_usd_ratio'] = df_features['borrow_total_usd'] / (df_features['deposit_total_usd'] + epsilon)
df_features['liquidated_ratio_count'] = df_features['liquidation_count'] / (df_features['tx_count'] + epsilon)
df_features['liquidated_ratio_usd'] = df_features['liquidation_total_usd'] / (df_features['borrow_total_usd'] + df_features['deposit_total_usd'] + epsilon)


In [10]:

# Bot-like Indicators (Heuristics)
df_features['tx_freq_per_day'] = df_features['tx_count'] / (df_features['account_lifespan_days'] + epsilon)
df_features['is_potentially_bot_time'] = ((df_features['std_time_between_tx_hours'] < 0.1) & (df_features['tx_count'] > 10)).astype(int) # Example threshold, store as 0/1
df_features['is_potentially_bot_dust'] = ((df_features['mean_tx_usd'] < 1.0) & (df_features['tx_count'] > 20)).astype(int) # Example threshold, store as 0/1


# --- Feature Cleaning/Refinement ---
df_features.replace([np.inf, -np.inf], 1e9, inplace=True) # Replace inf values

print("\nSample Features (Refactored):")
# Display relevant columns, including the ones that caused the error
print(df_features[['deposit_count', 'deposit_total_usd', 'borrow_count', 'borrow_total_usd', 'borrow_deposit_usd_ratio']].head())
print(f"\nTotal wallets with features: {len(df_features)}")


Sample Features (Refactored):
                                            deposit_count  deposit_total_usd  \
wallet_id                                                                      
0x00000000000067afd7fa546d3f63d4e53cdb8fa4              0                0.0   
0x0000000000007f150bd6f54c40a34d7c3d5e9f56              0                0.0   
0x00000000af5a61acaf76190794e3fdf1289288a1              0                0.0   
0x00000000fea1fab3ce021f3bedd21a94d0ba190c              0                0.0   
0x000000aaee6a496aaf7b7452518781786313400f              0                0.0   

                                            borrow_count  borrow_total_usd  \
wallet_id                                                                    
0x00000000000067afd7fa546d3f63d4e53cdb8fa4             0      0.000000e+00   
0x0000000000007f150bd6f54c40a34d7c3d5e9f56             0      0.000000e+00   
0x00000000af5a61acaf76190794e3fdf1289288a1             0      0.000000e+00   
0x00000000fea1fab3

In [11]:
from sklearn.preprocessing import MinMaxScaler

# --- Scoring Logic ---

# 1. Define feature contributions (positive/negative impact) and weights
#    Weights are subjective and need tuning based on intuition and observation.
feature_weights = {
    # Positive Factors (+)
    'account_lifespan_days': 0.10,         # Longer is better
    'deposit_total_usd': 0.15,             # Higher value supplied is good (use log transform?)
    'repay_total_usd': 0.10,               # Higher repayment value is good (log transform?)
    'repay_borrow_usd_ratio': 0.10,        # Ratio > 1 is good, cap needed?
    'unique_assets': 0.05,                 # Diversity is mildly positive
    'acted_liquidator_count': 0.05,        # Contributing to protocol health
    'net_flow_usd': 0.05,                  # Positive net flow is good (use log transform?)

    # Negative Factors (-) - Weights are positive, but subtracted from score
    'liquidated_count': 0.20,              # Strongest negative signal
    'liquidated_ratio_usd': 0.10,          # High proportion of value lost to liquidation is bad
    'borrow_deposit_usd_ratio': 0.05,      # High leverage is risky (penalize high values)
    'tx_freq_per_day': 0.03,               # Very high frequency can be bot-like/negative (penalize extremes)
    'is_potentially_bot_time': 0.01,       # Minor penalty for bot-like timing
    'is_potentially_bot_dust': 0.01,       # Minor penalty for dust transactions
}


In [12]:

# Normalize features (0 to 1) before applying weights
# Use log transform for highly skewed $ amount features to reduce outlier impact
skewed_usd_features = ['deposit_total_usd', 'repay_total_usd', 'net_flow_usd', 'borrow_total_usd', 'withdraw_total_usd', 'liquidated_total_usd', 'acted_liquidator_total_usd']
for feat in skewed_usd_features:
    # Add 1 before log to handle zeros (log(1)=0)
    # Ensure column exists before transforming
    if feat in df_features.columns:
         # Handle potential negative values in net_flow_usd if needed before log
        if feat == 'net_flow_usd':
            # Option 1: Scale differently, e.g. using StandardScaler or custom logic
            # Option 2: Separate positive/negative net flow?
            # Let's use clipping and scaling for simplicity here
            df_features[f'{feat}_log_scaled'] = np.log1p(df_features[feat].clip(lower=0))
        else:
            df_features[f'{feat}_log_scaled'] = np.log1p(df_features[feat])



In [13]:

# Select features for scaling (include log-transformed versions)
features_to_scale = list(feature_weights.keys()) + [f'{f}_log_scaled' for f in skewed_usd_features if f'{f}_log_scaled' in df_features.columns]
# Remove original USD features if log version exists and is used in weights
features_to_scale = [f for f in features_to_scale if f not in skewed_usd_features or f'{f}_log_scaled' not in df_features.columns]
# Add boolean bot flags back if they were removed
if 'is_potentially_bot_time' not in features_to_scale: features_to_scale.append('is_potentially_bot_time')
if 'is_potentially_bot_dust' not in features_to_scale: features_to_scale.append('is_potentially_bot_dust')

# Ensure all features actually exist in the dataframe
features_to_scale = [f for f in features_to_scale if f in df_features.columns]
# Remove duplicates
features_to_scale = list(dict.fromkeys(features_to_scale))


In [14]:


scaler = MinMaxScaler()
# Scale only the selected features used in the score calculation
scaled_features = scaler.fit_transform(df_features[features_to_scale])
df_scaled_features = pd.DataFrame(scaled_features, index=df_features.index, columns=features_to_scale)


# 2. Calculate Raw Score
df_features['raw_score'] = 50.0 # Start with a neutral base score

# Apply weights - adjust feature names if log-transformed versions are used
# Example: use 'deposit_total_usd_log_scaled' instead of 'deposit_total_usd'
positive_impact_features = {
    'account_lifespan_days': feature_weights['account_lifespan_days'],
    'deposit_total_usd_log_scaled': feature_weights['deposit_total_usd'], # Use log scaled
    'repay_total_usd_log_scaled': feature_weights['repay_total_usd'],     # Use log scaled
    'repay_borrow_usd_ratio': feature_weights['repay_borrow_usd_ratio'], # Maybe cap this feature? df_scaled_features['repay_borrow_usd_ratio'].clip(upper=5)
    'unique_assets': feature_weights['unique_assets'],
    'acted_liquidator_count': feature_weights['acted_liquidator_count'], # Maybe log scale this too if skewed?
    'net_flow_usd_log_scaled': feature_weights['net_flow_usd'],          # Use log scaled version for positive flows
}

negative_impact_features = {
    'liquidated_count': feature_weights['liquidated_count'], # Higher value = lower score
    'liquidated_ratio_usd': feature_weights['liquidated_ratio_usd'], # Higher value = lower score
    'borrow_deposit_usd_ratio': feature_weights['borrow_deposit_usd_ratio'], # Higher value = lower score (maybe cap?)
    'tx_freq_per_day': feature_weights['tx_freq_per_day'], # Higher value = lower score
    'is_potentially_bot_time': feature_weights['is_potentially_bot_time'], # Boolean (0 or 1) * weight
    'is_potentially_bot_dust': feature_weights['is_potentially_bot_dust'], # Boolean (0 or 1) * weight
}


In [15]:

# Sum weighted contributions (using scaled features)
for feat, weight in positive_impact_features.items():
    if feat in df_scaled_features.columns:
        # Increase score, scale weight effect (e.g., multiply by 50 so max contribution is 50*weight)
        df_features['raw_score'] += df_scaled_features[feat] * weight * 50
    else:
        print(f"Warning: Positive feature {feat} not found in scaled features.")


for feat, weight in negative_impact_features.items():
    if feat in df_scaled_features.columns:
         # Decrease score, scale weight effect (e.g., multiply by 50 so max penalty is 50*weight)
        df_features['raw_score'] -= df_scaled_features[feat] * weight * 50
    else:
        print(f"Warning: Negative feature {feat} not found in scaled features.")


# 3. Scale final score to 0-100 range
final_scaler = MinMaxScaler(feature_range=(0, 100))
df_features['credit_score'] = final_scaler.fit_transform(df_features[['raw_score']])

# Optional: Inspect score distribution
print("\nScore Distribution:")
print(df_features['credit_score'].describe())

# Optional: Clip scores just in case scaling resulted in tiny values outside 0-100 due to float precision
df_features['credit_score'] = df_features['credit_score'].clip(0, 100)


Score Distribution:
count    22715.000000
mean        12.618658
std          6.726509
min          0.000000
25%         10.053955
50%         10.053955
75%         11.513636
max        100.000000
Name: credit_score, dtype: float64


In [16]:
# Sort by score and select top 1000
top_1000_wallets = df_features.sort_values(by='credit_score', ascending=False).head(1000)

# Select only wallet_id and score for the final CSV
output_csv = top_1000_wallets[['credit_score']].reset_index()
output_csv.rename(columns={'index': 'wallet_id'}, inplace=True)

# Save to CSV
output_filename = 'top_1000_wallets.csv'
output_csv.to_csv(output_filename, index=False)
print(f"\nSaved top 1,000 wallets to {output_filename}")


Saved top 1,000 wallets to top_1000_wallets.csv


# Final Pipeline of the entire setup and analysis

In [17]:
# model_pipeline.py

import json
import pandas as pd
import numpy as np
from datetime import datetime
import os
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings

# Suppress specific warnings if needed (e.g., for pandas apply)
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# warnings.filterwarnings("ignore", category=FutureWarning)

print("--- Starting Compound V2 Wallet Scoring Pipeline ---")

# --- Configuration ---
# Assume data is in a 'data' subdirectory relative to the script
DATA_DIR = 'data/'
OUTPUT_DIR = 'outputs/'
# Create output dir if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Automatically find JSON files in the data directory
# In a real scenario, you'd implement logic to select the 3 largest files.
# For this example, let's assume we manually list them or just process all .json files found.
# json_files = glob.glob(os.path.join(DATA_DIR, '*.json'))
# Example: Manually list the 3 largest if needed after checking sizes
# files_to_process = [os.path.join(DATA_DIR, 'file1.json'), os.path.join(DATA_DIR, 'file2.json'), os.path.join(DATA_DIR, 'file3.json')]
# files_to_process = json_files # Process all found JSON files for this example
print(f"Found files to process: {files_to_process}")

if not files_to_process:
    print(f"Error: No JSON files found in {DATA_DIR}. Exiting.")
    exit()

# --- 1. Data Loading Function ---
def load_data(filepath):
    """Loads transaction data from a single JSON file."""
    try:
        with open(filepath, 'r') as f:
            # Handle potential multiple JSON objects per file if needed
            # For simplicity, assume one top-level object per file
            data = json.load(f)
        print(f"Successfully loaded {filepath}")
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from {filepath}: {e}")
        return None
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return None

# --- 2. Data Processing Function ---
def process_raw_data(raw_data_list):
    """Processes raw data from multiple files into a single DataFrame."""
    all_tx = []
    # Define expected transaction types based on sample data
    valid_tx_keys = ['deposits', 'borrows', 'repayments', 'withdrawals', 'liquidations']

    for file_index, data in enumerate(raw_data_list):
        if data is None:
            print(f"Skipping empty or invalid data from file index {file_index}")
            continue

        # Check if data is a dictionary as expected
        if not isinstance(data, dict):
            print(f"Warning: Data from file index {file_index} is not a dictionary (found {type(data)}). Skipping.")
            continue

        for tx_key, transactions in data.items():
            # Check if the key is one we expect and if the value is a list
            if tx_key not in valid_tx_keys:
                print(f"Warning: Unexpected key '{tx_key}' in file index {file_index}. Skipping.")
                continue

            if not isinstance(transactions, list):
                print(f"Warning: Expected list for '{tx_key}', got {type(transactions)}. Skipping.")
                continue

            # Normalize key name
            normalized_tx_type = tx_key.lower().replace('s', '')
            if 'repayment' in normalized_tx_type: normalized_tx_type = 'repay'
            if 'withdrawal' in normalized_tx_type: normalized_tx_type = 'withdraw'

            for tx in transactions:
                try:
                    record = {
                        'tx_hash': tx.get('hash'),
                        'log_id': tx.get('id'),
                        'timestamp_unix': int(tx.get('timestamp')),
                        'tx_type': normalized_tx_type,
                        'wallet_id': tx.get('account', {}).get('id'),
                        'asset_symbol': tx.get('asset', {}).get('symbol'),
                        'asset_id': tx.get('asset', {}).get('id'),
                        'amount': float(tx.get('amount', 0)),
                        'amountUSD': float(tx.get('amountUSD', 0)),
                        'liquidator': None,
                        'liquidated_account': None
                    }

                    if normalized_tx_type == 'liquidation':
                        record['liquidator'] = tx.get('liquidator', {}).get('id')
                        record['liquidated_account'] = tx.get('account', {}).get('id')
                        # Override wallet_id to the account being liquidated for primary aggregation
                        record['wallet_id'] = record['liquidated_account']

                    # Basic validation
                    if record['wallet_id'] and record['timestamp_unix'] is not None and record['amountUSD'] >= 0:
                         # Convert timestamp here
                        record['timestamp'] = pd.to_datetime(record['timestamp_unix'], unit='s')
                        all_tx.append(record)
                    else:
                         print(f"Skipping record due to missing/invalid data: ID {tx.get('id', 'N/A')}")

                except (TypeError, ValueError) as e:
                    print(f"Error processing record {tx.get('id', 'N/A')} in {tx_key}: {e}. Skipping record.")
                except Exception as e:
                    print(f"Unexpected error processing record {tx.get('id', 'N/A')} in {tx_key}: {e}. Skipping record.")


    if not all_tx:
        print("Error: No valid transactions processed. Exiting.")
        exit()

    df = pd.DataFrame(all_tx)
    df = df.sort_values(by='timestamp').reset_index(drop=True)
    print(f"Processed {len(df)} total valid transactions.")
    return df

# --- 3. Feature Engineering Function ---
def engineer_features(df_tx):
    """Engineers features for each wallet."""
    print("Starting feature engineering...")
    if df_tx.empty:
        print("Cannot engineer features: Input DataFrame is empty.")
        return pd.DataFrame()

    # Group by wallet
    wallet_groups = df_tx.groupby('wallet_id')

    # Calculate basic aggregations
    features_agg = wallet_groups.agg(
        tx_count=('tx_hash', 'count'),
        unique_assets=('asset_symbol', 'nunique'),
        first_tx_time=('timestamp', 'min'),
        last_tx_time=('timestamp', 'max'),
        mean_tx_usd=('amountUSD', 'mean')
    )

    # Calculate type-specific counts and sums
    tx_type_counts = df_tx.groupby(['wallet_id', 'tx_type']).size().unstack(fill_value=0)
    tx_type_sums_usd = df_tx.groupby(['wallet_id', 'tx_type'])['amountUSD'].sum().unstack(fill_value=0)

    tx_type_counts.columns = [f"{col}_count" for col in tx_type_counts.columns]
    tx_type_sums_usd.columns = [f"{col}_total_usd" for col in tx_type_sums_usd.columns]

    # Combine aggregated features
    df_features = pd.concat([features_agg, tx_type_counts, tx_type_sums_usd], axis=1)

    # Ensure all expected columns exist, even if they are all zero
    expected_tx_types = ['deposit', 'borrow', 'repay', 'withdraw', 'liquidation']
    for ttype in expected_tx_types:
        count_col = f"{ttype}_count"
        sum_col = f"{ttype}_total_usd"
        if count_col not in df_features.columns: df_features[count_col] = 0
        if sum_col not in df_features.columns: df_features[sum_col] = 0.0

    df_features.fillna(0, inplace=True) # Fill any remaining NaNs

    # Calculate derived features
    df_features['account_lifespan_days'] = (df_features['last_tx_time'] - df_features['first_tx_time']).dt.total_seconds() / (60 * 60 * 24)
    df_features['account_lifespan_days'] = df_features['account_lifespan_days'].apply(lambda x: max(x, 1/(24*60))) # Min 1 min lifespan

    # Liquidation actor features
    liquidator_actions = df_tx[df_tx['tx_type'] == 'liquidation'].groupby('liquidator').agg(
        acted_liquidator_count=('tx_hash', 'count'),
        acted_liquidator_total_usd=('amountUSD', 'sum')
    ).rename_axis('wallet_id') # Rename index to merge correctly
    df_features = df_features.merge(liquidator_actions, left_index=True, right_index=True, how='left')
    df_features[['acted_liquidator_count', 'acted_liquidator_total_usd']] = df_features[['acted_liquidator_count', 'acted_liquidator_total_usd']].fillna(0)

    # Time-based features (using apply, may show warning)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=FutureWarning) # Suppress specific apply warning
        df_features['avg_time_between_tx_hours'] = wallet_groups['timestamp'].apply(
            lambda x: x.diff().mean().total_seconds() / 3600 if len(x) > 1 else 0
        ).fillna(0)
        df_features['std_time_between_tx_hours'] = wallet_groups['timestamp'].apply(
            lambda x: x.diff().std().total_seconds() / 3600 if len(x) > 1 else 0
        ).fillna(0)

    # Ratios and Indicators
    epsilon = 1e-6
    df_features['repay_borrow_count_ratio'] = df_features['repay_count'] / (df_features['borrow_count'] + epsilon)
    df_features['repay_borrow_usd_ratio'] = df_features['repay_total_usd'] / (df_features['borrow_total_usd'] + epsilon)
    df_features['deposit_withdraw_usd_ratio'] = df_features['deposit_total_usd'] / (df_features['withdraw_total_usd'] + epsilon)
    df_features['net_flow_usd'] = (df_features['deposit_total_usd'] + df_features['repay_total_usd']) - \
                                  (df_features['borrow_total_usd'] + df_features['withdraw_total_usd'] + df_features['liquidation_total_usd'])
    df_features['borrow_deposit_usd_ratio'] = df_features['borrow_total_usd'] / (df_features['deposit_total_usd'] + epsilon)
    df_features['liquidated_ratio_count'] = df_features['liquidation_count'] / (df_features['tx_count'] + epsilon)
    df_features['liquidated_ratio_usd'] = df_features['liquidation_total_usd'] / (df_features['borrow_total_usd'] + df_features['deposit_total_usd'] + epsilon)
    df_features['tx_freq_per_day'] = df_features['tx_count'] / (df_features['account_lifespan_days'] + epsilon)
    df_features['is_potentially_bot_time'] = ((df_features['std_time_between_tx_hours'] < 0.1) & (df_features['tx_count'] > 10)).astype(int)
    df_features['is_potentially_bot_dust'] = ((df_features['mean_tx_usd'] < 1.0) & (df_features['tx_count'] > 20)).astype(int)

    # Replace inf values just in case
    df_features.replace([np.inf, -np.inf], 1e12, inplace=True) # Use a large number instead of inf

    # Log transform skewed USD features (add column with _log suffix)
    skewed_usd_features = ['deposit_total_usd', 'repay_total_usd', 'borrow_total_usd', 'withdraw_total_usd', 'liquidation_total_usd', 'acted_liquidator_total_usd', 'net_flow_usd']
    for feat in skewed_usd_features:
        if feat in df_features.columns:
            if feat == 'net_flow_usd':
                # Handle potential negative values: scale shifted log
                min_net_flow = df_features[feat].min()
                df_features[f'{feat}_log'] = np.log1p(df_features[feat] - min_net_flow + epsilon) # Shift so min is >= 0
            else:
                 # Ensure non-negative before log1p
                df_features[f'{feat}_log'] = np.log1p(df_features[feat].clip(lower=0))
        else:
             print(f"Warning: Skewed feature {feat} not found for log transform.")


    print(f"Feature engineering complete. Generated {len(df_features.columns)} features for {len(df_features)} wallets.")
    return df_features

# --- 4. Scoring Function ---
def calculate_scores(df_features):
    """Calculates credit scores based on engineered features."""
    print("Calculating credit scores...")
    if df_features.empty:
        print("Cannot calculate scores: Input DataFrame is empty.")
        return df_features

    # Define feature contributions and weights (Adjust these based on domain knowledge/analysis)
    # Using log-transformed versions for amounts where available
    feature_weights = {
        # Positive Factors (+) - Higher value = Higher score contribution
        'account_lifespan_days': 0.08,
        'deposit_total_usd_log': 0.15,
        'repay_total_usd_log': 0.10,
        'repay_borrow_usd_ratio': 0.12, # Cap this later?
        'unique_assets': 0.03,
        'acted_liquidator_count': 0.05, # Maybe log transform count?
        'net_flow_usd_log': 0.05,
        'deposit_withdraw_usd_ratio': 0.02, # Mildly positive if > 1

        # Negative Factors (-) - Higher value = Lower score contribution (weights are positive magnitude)
        'liquidation_count': 0.20,       # Very important penalty
        'liquidated_ratio_usd': 0.10,    # High proportion liquidated is bad
        'borrow_deposit_usd_ratio': 0.06,# High leverage is risky
        'tx_freq_per_day': 0.02,         # Very high freq might be negative
        'is_potentially_bot_time': 0.01,
        'is_potentially_bot_dust': 0.01,
    }

    # Select features for scaling (make sure they exist in df_features)
    features_to_scale = [f for f in feature_weights.keys() if f in df_features.columns]

    # Cap extreme ratio features before scaling to prevent excessive influence
    if 'repay_borrow_usd_ratio' in df_features.columns:
        df_features['repay_borrow_usd_ratio_capped'] = df_features['repay_borrow_usd_ratio'].clip(upper=10) # Example cap
        if 'repay_borrow_usd_ratio_capped' not in features_to_scale: features_to_scale.append('repay_borrow_usd_ratio_capped')
        if 'repay_borrow_usd_ratio' in features_to_scale: features_to_scale.remove('repay_borrow_usd_ratio') # Use capped version

    if 'borrow_deposit_usd_ratio' in df_features.columns:
        df_features['borrow_deposit_usd_ratio_capped'] = df_features['borrow_deposit_usd_ratio'].clip(upper=20) # Example cap (20x leverage is huge)
        if 'borrow_deposit_usd_ratio_capped' not in features_to_scale: features_to_scale.append('borrow_deposit_usd_ratio_capped')
        if 'borrow_deposit_usd_ratio' in features_to_scale: features_to_scale.remove('borrow_deposit_usd_ratio') # Use capped version

    if 'deposit_withdraw_usd_ratio' in df_features.columns:
        df_features['deposit_withdraw_usd_ratio_capped'] = df_features['deposit_withdraw_usd_ratio'].clip(upper=10) # Example cap
        if 'deposit_withdraw_usd_ratio_capped' not in features_to_scale: features_to_scale.append('deposit_withdraw_usd_ratio_capped')
        if 'deposit_withdraw_usd_ratio' in features_to_scale: features_to_scale.remove('deposit_withdraw_usd_ratio') # Use capped version

    if 'tx_freq_per_day' in df_features.columns:
        df_features['tx_freq_per_day_capped'] = df_features['tx_freq_per_day'].clip(upper=df_features['tx_freq_per_day'].quantile(0.99)) # Cap at 99th percentile
        if 'tx_freq_per_day_capped' not in features_to_scale: features_to_scale.append('tx_freq_per_day_capped')
        if 'tx_freq_per_day' in features_to_scale: features_to_scale.remove('tx_freq_per_day') # Use capped version


    # Scale only the selected (and potentially capped) features
    scaler = MinMaxScaler()
    scaled_features_data = scaler.fit_transform(df_features[features_to_scale])
    df_scaled_features = pd.DataFrame(scaled_features_data, index=df_features.index, columns=features_to_scale)

    # Calculate Raw Score
    df_features['raw_score'] = 50.0 # Start neutral

    positive_features = [
        'account_lifespan_days', 'deposit_total_usd_log', 'repay_total_usd_log',
        'repay_borrow_usd_ratio_capped', 'unique_assets', 'acted_liquidator_count',
        'net_flow_usd_log', 'deposit_withdraw_usd_ratio_capped'
    ]
    negative_features = [
        'liquidation_count', 'liquidated_ratio_usd', 'borrow_deposit_usd_ratio_capped',
        'tx_freq_per_day_capped', 'is_potentially_bot_time', 'is_potentially_bot_dust'
    ]

    # Sum weighted contributions (using scaled features)
    total_weight_applied = 0
    for feat, weight in feature_weights.items():
        # Determine the actual feature name used (original or capped/log version)
        scaled_feat_name = feat
        if feat + '_capped' in df_scaled_features.columns:
            scaled_feat_name = feat + '_capped'
        elif feat + '_log' in df_scaled_features.columns:
             scaled_feat_name = feat + '_log'

        if scaled_feat_name in df_scaled_features.columns:
            if feat in positive_features:
                df_features['raw_score'] += df_scaled_features[scaled_feat_name] * weight * 50 # Max contribution = weight * 50
            elif feat in negative_features:
                df_features['raw_score'] -= df_scaled_features[scaled_feat_name] * weight * 50 # Max penalty = weight * 50
            total_weight_applied += weight
        else:
            print(f"Warning: Feature '{feat}' (or derived version) not found in scaled features for scoring.")

    print(f"Applied scoring logic using {len(features_to_scale)} features with total weight magnitude: {total_weight_applied:.2f}")

    # Scale final score to 0-100 range
    final_scaler = MinMaxScaler(feature_range=(0, 100))
    df_features['credit_score'] = final_scaler.fit_transform(df_features[['raw_score']])
    df_features['credit_score'] = df_features['credit_score'].round(2) # Round for cleaner output

    print("Credit scoring complete.")
    return df_features


# --- Main Execution ---
# 1. Load Data
print("--- Phase 1: Loading Data ---")
raw_data_loaded = [load_data(f_path) for f_path in files_to_process]
raw_data_loaded = [d for d in raw_data_loaded if d is not None] # Filter out failed loads

if not raw_data_loaded:
    print("Error: No data could be loaded. Exiting.")
    exit()

# 2. Process Data
print("\n--- Phase 2: Processing Raw Data ---")
df_tx = process_raw_data(raw_data_loaded)

# 3. Engineer Features
print("\n--- Phase 3: Engineering Features ---")
df_features = engineer_features(df_tx)

# 4. Calculate Scores
print("\n--- Phase 4: Calculating Scores ---")
df_scored_features = calculate_scores(df_features)

# --- 5. Output Generation ---
print("\n--- Phase 5: Generating Outputs ---")

# Save the full features and scores DataFrame (useful for Streamlit app and analysis)
full_output_path = os.path.join(OUTPUT_DIR, 'wallet_scores_features.csv')
try:
    df_scored_features.reset_index().to_csv(full_output_path, index=False)
    print(f"Successfully saved full features and scores to {full_output_path}")
except Exception as e:
    print(f"Error saving full features CSV: {e}")

# Save the Top 1,000 Wallets CSV
top_1000_output_path = os.path.join(OUTPUT_DIR, 'top_1000_wallets.csv')
try:
    top_1000_wallets = df_scored_features.sort_values(by='credit_score', ascending=False).head(1000)
    # Select only wallet_id and score
    top_1000_output_csv = top_1000_wallets[['credit_score']].reset_index()
    # top_1000_output_csv.rename(columns={'index': 'wallet_id'}, inplace=True) # Index name is already 'wallet_id'
    top_1000_output_csv.to_csv(top_1000_output_path, index=False)
    print(f"Successfully saved top 1,000 wallets to {top_1000_output_path}")
except Exception as e:
    print(f"Error saving top 1000 wallets CSV: {e}")


# --- Generate Wallet Analysis Snippets (for the separate document) ---
print("\n--- Generating Wallet Analysis Snippets ---")
analysis_output_path = os.path.join(OUTPUT_DIR, 'wallet_analysis_snippets.txt')
try:
    with open(analysis_output_path, 'w') as f:
        f.write("Wallet Analysis (Top 5 High / Top 5 Low Scores)\n")
        f.write("="*50 + "\n\n")

        top_5 = df_scored_features.sort_values(by='credit_score', ascending=False).head(5)
        low_5 = df_scored_features.sort_values(by='credit_score', ascending=True).head(5)

        f.write("--- Top 5 High-Scoring Wallets ---\n")
        for wallet_id, data in top_5.iterrows():
            f.write(f"\nWallet ID: {wallet_id}\n")
            f.write(f"Score: {data['credit_score']:.2f}\n")
            f.write(f"Key Features:\n")
            f.write(f"  - Lifespan (Days): {data.get('account_lifespan_days', 'N/A'):.1f}\n")
            f.write(f"  - Deposit USD (Log): {data.get('deposit_total_usd_log', 'N/A'):.2f}\n")
            f.write(f"  - Repay/Borrow USD Ratio: {data.get('repay_borrow_usd_ratio', 'N/A'):.2f} (Capped: {data.get('repay_borrow_usd_ratio_capped', 'N/A'):.2f})\n")
            f.write(f"  - Liquidated Count: {int(data.get('liquidation_count', 0))}\n")
            f.write(f"  - Acted Liquidator Count: {int(data.get('acted_liquidator_count', 0))}\n")
            f.write(f"  - Borrow/Deposit USD Ratio: {data.get('borrow_deposit_usd_ratio', 'N/A'):.2f} (Capped: {data.get('borrow_deposit_usd_ratio_capped', 'N/A'):.2f})\n")
            # Add more features as needed
            f.write("Justification: High score likely due to significant deposits/repayments, long lifespan, potentially acting as liquidator, and absence of liquidations.\n")


        f.write("\n--- Top 5 Low-Scoring Wallets ---\n")
        for wallet_id, data in low_5.iterrows():
            f.write(f"\nWallet ID: {wallet_id}\n")
            f.write(f"Score: {data['credit_score']:.2f}\n")
            f.write(f"Key Features:\n")
            f.write(f"  - Lifespan (Days): {data.get('account_lifespan_days', 'N/A'):.1f}\n")
            f.write(f"  - Deposit USD (Log): {data.get('deposit_total_usd_log', 'N/A'):.2f}\n")
            f.write(f"  - Repay/Borrow USD Ratio: {data.get('repay_borrow_usd_ratio', 'N/A'):.2f} (Capped: {data.get('repay_borrow_usd_ratio_capped', 'N/A'):.2f})\n")
            f.write(f"  - Liquidated Count: {int(data.get('liquidation_count', 0))}\n")
            f.write(f"  - Liquidated Ratio USD: {data.get('liquidated_ratio_usd', 'N/A'):.4f}\n")
            f.write(f"  - Acted Liquidator Count: {int(data.get('acted_liquidator_count', 0))}\n")
            f.write(f"  - Borrow/Deposit USD Ratio: {data.get('borrow_deposit_usd_ratio', 'N/A'):.2f} (Capped: {data.get('borrow_deposit_usd_ratio_capped', 'N/A'):.2f})\n")
            # Add more features as needed
            f.write("Justification: Low score likely driven by liquidations (high count or high ratio), high leverage (borrow/deposit ratio), or potentially short lifespan/low activity.\n")

    print(f"Successfully generated wallet analysis snippets to {analysis_output_path}")
except Exception as e:
    print(f"Error generating wallet analysis snippets: {e}")


print("\n--- Pipeline Finished ---")

--- Starting Compound V2 Wallet Scoring Pipeline ---
Found files to process: ['/content/compoundV2_transactions_ethereum_chunk_0.json', '/content/compoundV2_transactions_ethereum_chunk_1.json', '/content/compoundV2_transactions_ethereum_chunk_32.json']
--- Phase 1: Loading Data ---
Successfully loaded /content/compoundV2_transactions_ethereum_chunk_0.json
Successfully loaded /content/compoundV2_transactions_ethereum_chunk_1.json
Successfully loaded /content/compoundV2_transactions_ethereum_chunk_32.json

--- Phase 2: Processing Raw Data ---
Processed 50000 total valid transactions.

--- Phase 3: Engineering Features ---
Starting feature engineering...
Feature engineering complete. Generated 39 features for 16274 wallets.

--- Phase 4: Calculating Scores ---
Calculating credit scores...
Applied scoring logic using 14 features with total weight magnitude: 1.00
Credit scoring complete.

--- Phase 5: Generating Outputs ---
Successfully saved full features and scores to outputs/wallet_score